In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import evaluate
import numpy as np

from pprint import pprint
from datasets import load_dataset
from sklearn.cluster import KMeans

from transformers import PegasusTokenizer, PegasusForConditionalGeneration
from transformers import BertTokenizer, BertModel
from nltk.tokenize import sent_tokenize

# sure the GPUs are working 
import tensorflow as tf

# Check for TensorFlow GPU access
print(f"TensorFlow has access to the following devices:\n{tf.config.list_physical_devices()}")

# See TensorFlow version
print(f"TensorFlow version: {tf.__version__}")


# # Load the dataset
# dataset = load_dataset("allenai/mslr2022", "ms2", split='validation')

# # Convert the dataset to a Pandas DataFrame
# df = dataset.to_pandas()


# Load the data- test inputs
df = pd.read_csv('mslr2022_validation.csv')


df = df[['review_id', 'abstract']]

#save the first 10 rows for practice 
df= df[5:10]

# df.head()

TensorFlow has access to the following devices:
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
TensorFlow version: 2.9.0


In [ ]:
df.head(15)

In [ ]:
# Split the abstracts into lists of sentences
df['split_abstracts'] = df['abstract'].apply(lambda x: x.split('\n '))
df['split_abstracts']

***Extractive step: SCiBERT, K means***

In [14]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import torch
from transformers import BertTokenizer, BertModel

# Assuming df is your actual DataFrame with 'review_id' and 'abstract' columns

# Split the concatenated abstracts into lists of abstracts
df['split_abstracts'] = df['abstract'].apply(lambda x: x.split('\n '))

# Initialize SciBERT
tokenizer = BertTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
model = BertModel.from_pretrained('allenai/scibert_scivocab_uncased')

def bert_sentence_embeddings(sentences):
    embeddings = []
    for sentence in sentences:
        inputs = tokenizer(sentence, return_tensors='pt', max_length=512, truncation=True)
        with torch.no_grad():
            outputs = model(**inputs)
        embeddings.append(outputs.last_hidden_state.mean(dim=1).squeeze().numpy())
    return np.array(embeddings)

def select_top_sentences(sentences, embeddings, n_sentences=5):
    if len(sentences) < n_sentences:
        return ' '.join(sentences)
    kmeans = KMeans(n_clusters=n_sentences, n_init=10)
    kmeans.fit(embeddings)
    top_sentence_indices = np.argmin(
        np.linalg.norm(embeddings[:, np.newaxis] - kmeans.cluster_centers_, axis=2), axis=0)
    top_sentences = [sentences[index] for index in top_sentence_indices]
    return ' '.join(top_sentences)

# Process each abstract and create summaries
summaries = {}
for _, row in df.iterrows():
    review_id = row['review_id']
    split_abstracts = row['split_abstracts']
    combined_summary = ''
    
    for abstract in split_abstracts:
        # Split each abstract into sentences
        sentences = abstract.split('. ')  # Assuming sentences in abstracts are separated by '. '
        # Generate embeddings for each sentence
        embeddings = bert_sentence_embeddings(sentences)
        # Select the top sentences from these embeddings
        summary = select_top_sentences(sentences, embeddings)
        # Combine the summaries from each abstract
        combined_summary += summary + ' '
    
    # Add the combined summary for the current review_id to the summaries dictionary
    summaries[review_id] = combined_summary.strip()

# Print the summaries for each review_id
for review_id, summary in summaries.items():
    print(f"Review ID: {review_id}\nSummary: {summary}\n")


Review ID: 16801507
Summary: Markers of bone metabolism , including serum OPG , osteocalcin ( OC ) , bone-specific alkaline phosphatase activity ( BAP ) , PTH , tartrate-resistant acid phosphatase ( TRAP ) , and bone mineral density ( BMD ) , were measured before and during the treatment  BAP had decreased at month 3 in group A ( P < 0.05 ) , but not in group B Twenty patients with chronic glomerulonephritis treated with GC for the first time were chosen for this study  OC was decreased at month 1 but subsequently increased until month 12 in both groups  The present study demonstrated that the inhibition exerted by vitamin K2 of the reduction in OPG induced by GC may , at least in part , play a role in the prevention and treatment of GC-induced  bone loss' There were no significant differences between the groups in the incidence of serious adverse events  METHODS To evaluate the efficacy of strontium ranelate in preventing vertebral fractures in a phase 3 trial , we r and omly assigned

In [5]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

# Load pre-trained Pegasus tokenizer and model
tokenizer_pegasus = PegasusTokenizer.from_pretrained('google/pegasus-xsum')
model_pegasus = PegasusForConditionalGeneration.from_pretrained('google/pegasus-xsum')

# Dictionary to store the final summaries
final_summaries = {}

# Iterate over each summary in the summaries dictionary
for review_id, extractive_summary in summaries.items():
    # Prepare the input for the model
    inputs = tokenizer_pegasus(
        extractive_summary, 
        truncation=True, 
        padding="longest", 
        return_tensors="pt", 
        max_length=512
    )

    # Generate the summary with Pegasus
    try:
        summary_ids = model_pegasus.generate(
            inputs['input_ids'], 
            num_beams=4,
            min_length=50,
            max_length=200,             
            length_penalty=2.0, 
            early_stopping=True
        )
        
        # Decode the generated IDs to text
        pegasus_summary = tokenizer_pegasus.decode(
            summary_ids[0], 
            skip_special_tokens=True
        )
        
        # Store the summary in the final summaries dictionary
        final_summaries[review_id] = pegasus_summary

    except IndexError as e:
        print(f"Error processing review_id {review_id}: {e}")
        final_summaries[review_id] = ""

# Display the final summaries
for review_id, summary in final_summaries.items():
    print(f"Review ID: {review_id}\nAbstractive Summary: {summary}\n")


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Review ID: 16801507
Abstractive Summary: glomeruloneitis ( GC ) has been shown to cause bone loss in chronic glomeruloneitis patients and to be a risk factor for fractures in postmenopausal women, but little is known about the mechanism of action of GC on bone metabolism and its effect on bone mineral density.

Review ID: 11676811
Abstractive Summary: The purpose of this study was to investigate the effects of two different exercise programs on low back problems in patients after 1year supervision by a physiotherapist and to assess the efficacy of lumbar supports and education in the prevention of low back pain in industry.

Review ID: 12719681
Abstractive Summary: The quality of life of patients with chronic obstructive pulmonary disease ( COPD) has been investigated in a series of clinical trials and reports have been published in the British Journal of Respiratory Medicine and the International Journal of Sports Medicine and Sport Science, as well as in the journal of the American C

____

***BioBERT***

In [8]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import torch
from transformers import AutoTokenizer, AutoModel


# Assuming df is your actual DataFrame with 'review_id' and 'abstract' columns

# Split the concatenated abstracts into lists of abstracts
df['split_abstracts'] = df['abstract'].apply(lambda x: x.split('\n '))

# Load the BioBERT tokenizer and model
model_name = "dmis-lab/biobert-base-cased-v1.2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)


def bert_sentence_embeddings(sentences):
    embeddings = []
    for sentence in sentences:
        inputs = tokenizer(sentence, return_tensors='pt', max_length=512, truncation=True)
        with torch.no_grad():
            outputs = model(**inputs)
        embeddings.append(outputs.last_hidden_state.mean(dim=1).squeeze().numpy())
    return np.array(embeddings)

def select_top_sentences(sentences, embeddings, n_sentences=5):
    if len(sentences) < n_sentences:
        return ' '.join(sentences)
    kmeans = KMeans(n_clusters=n_sentences, n_init=10)
    kmeans.fit(embeddings)
    top_sentence_indices = np.argmin(
        np.linalg.norm(embeddings[:, np.newaxis] - kmeans.cluster_centers_, axis=2), axis=0)
    top_sentences = [sentences[index] for index in top_sentence_indices]
    return ' '.join(top_sentences)

# Process each abstract and create summaries
summaries = {}
for _, row in df.iterrows():
    review_id = row['review_id']
    split_abstracts = row['split_abstracts']
    combined_summary = ''
    
    for abstract in split_abstracts:
        # Split each abstract into sentences
        sentences = abstract.split('. ')  # Assuming sentences in abstracts are separated by '. '
        # Generate embeddings for each sentence
        embeddings = bert_sentence_embeddings(sentences)
        # Select the top sentences from these embeddings
        summary = select_top_sentences(sentences, embeddings)
        # Combine the summaries from each abstract
        combined_summary += summary + ' '
    
    # Add the combined summary for the current review_id to the summaries dictionary
    summaries[review_id] = combined_summary.strip()

# # Print the summaries for each review_id
# for review_id, summary in summaries.items():
#     print(f"Review ID: {review_id}\nSummary: {summary}\n")



##########


from transformers import PegasusForConditionalGeneration, PegasusTokenizer

# Load pre-trained Pegasus tokenizer and model
tokenizer_pegasus = PegasusTokenizer.from_pretrained('google/pegasus-xsum')
model_pegasus = PegasusForConditionalGeneration.from_pretrained('google/pegasus-xsum')

# Dictionary to store the final summaries
final_summaries = {}

# Iterate over each summary in the summaries dictionary
for review_id, extractive_summary in summaries.items():
    # Prepare the input for the model
    inputs = tokenizer_pegasus(
        extractive_summary, 
        truncation=True, 
        padding="longest", 
        return_tensors="pt", 
        max_length=512
    )

    # Generate the summary with Pegasus
    try:
        summary_ids = model_pegasus.generate(
            inputs['input_ids'], 
            num_beams=4,
            min_length=50,
            max_length=200,             
            length_penalty=2.0, 
            early_stopping=True
        )
        
        # Decode the generated IDs to text
        pegasus_summary = tokenizer_pegasus.decode(
            summary_ids[0], 
            skip_special_tokens=True
        )
        
        # Store the summary in the final summaries dictionary
        final_summaries[review_id] = pegasus_summary

    except IndexError as e:
        print(f"Error processing review_id {review_id}: {e}")
        final_summaries[review_id] = ""

# Display the final summaries
for review_id, summary in final_summaries.items():
    print(f"Review ID: {review_id}\nAbstractive Summary: {summary}\n")




Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Review ID: 16801507
Abstractive Summary: The present study was carried out to evaluate the effect of glomerulonephritis ( GC ) on bone loss in patients with chronic glomerulonephritis and to assess the efficacy of strontium ranelate in preventing vertebral fractures in postmenopausal women.

Review ID: 11676811
Abstractive Summary: The aim of this study was to compare the effectiveness of lumbar supports and education for the prevention of low back pain in patients after a factorial design with the use of randomised controlled trials (RCTs) and historical controls (HCTs) for clinical trials.

Review ID: 12719681
Abstractive Summary: Quality of life (QOL) scores predict re-admission for COPD or death within 12 months of an original admission, and predict home nebuliser provision in a study of 377 patients admitted with an exacerbation of COPD, with 111 of whom were not included in the study because they refused the interview or died before discharge.

Review ID: 26830881
Abstractive Sum

___

***BioBERT and T5***

In [11]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import torch
from transformers import AutoTokenizer, AutoModel, T5Tokenizer, T5ForConditionalGeneration

# Assuming df is your actual DataFrame with 'review_id' and 'abstract' columns

# Split the concatenated abstracts into lists of abstracts
df['split_abstracts'] = df['abstract'].apply(lambda x: x.split('\n '))

# Load the BioBERT tokenizer and model
biobert_model_name = "dmis-lab/biobert-base-cased-v1.2"
tokenizer_biobert = AutoTokenizer.from_pretrained(biobert_model_name)
model_biobert = AutoModel.from_pretrained(biobert_model_name)

def bert_sentence_embeddings(sentences):
    embeddings = []
    for sentence in sentences:
        inputs = tokenizer_biobert(sentence, return_tensors='pt', max_length=512, truncation=True)
        with torch.no_grad():
            outputs = model_biobert(**inputs)
        embeddings.append(outputs.last_hidden_state.mean(dim=1).squeeze().numpy())
    return np.array(embeddings)

def select_top_sentences(sentences, embeddings, n_sentences=5):
    if len(sentences) < n_sentences:
        return ' '.join(sentences)
    kmeans = KMeans(n_clusters=n_sentences, n_init=10)
    kmeans.fit(embeddings)
    top_sentence_indices = np.argmin(
        np.linalg.norm(embeddings[:, np.newaxis] - kmeans.cluster_centers_, axis=2), axis=0)
    top_sentences = [sentences[index] for index in top_sentence_indices]
    return ' '.join(top_sentences)

# Process each abstract and create summaries
summaries = {}
for _, row in df.iterrows():
    review_id = row['review_id']
    split_abstracts = row['split_abstracts']
    combined_summary = ''
    
    for abstract in split_abstracts:
        sentences = abstract.split('. ')  # Assuming sentences in abstracts are separated by '. '
        embeddings = bert_sentence_embeddings(sentences)
        summary = select_top_sentences(sentences, embeddings)
        combined_summary += summary + ' '
    
    summaries[review_id] = combined_summary.strip()

##########

# Load pre-trained T5 tokenizer and model
t5_model_name = "t5-small"  # You can choose other versions like 't5-base', 't5-large'
tokenizer_t5 = T5Tokenizer.from_pretrained(t5_model_name)
model_t5 = T5ForConditionalGeneration.from_pretrained(t5_model_name)

# Dictionary to store the final summaries
final_summaries = {}

for review_id, extractive_summary in summaries.items():
    # Prepend the prompt for summarization task
    prompt = "summarize: " + extractive_summary
    
    inputs = tokenizer_t5(
        prompt, 
        return_tensors="pt", 
        max_length=512, 
        truncation=True, 
        padding="longest"
    )

    try:
        summary_ids = model_t5.generate(
            inputs['input_ids'],
            num_beams=4,
            min_length=50,
            max_length=200,
            length_penalty=2.0,
            early_stopping=True
        )
        
        t5_summary = tokenizer_t5.decode(
            summary_ids[0], 
            skip_special_tokens=True
        )
        
        final_summaries[review_id] = t5_summary

    except IndexError as e:
        print(f"Error processing review_id {review_id}: {e}")
        final_summaries[review_id] = ""

for review_id, summary in final_summaries.items():
    print(f"Review ID: {review_id}\nAbstractive Summary: {summary}\n")


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Review ID: 16801507
Abstractive Summary: serum OPG, osteocalcin ( OC ), bone-specific alkaline phosphatase activity ( BAP ), PTH, tartrate-resistant acid phosphatase ( TRAP ), and bone mineral density ( BMD ) were measured before and during the treatment. no significant change was seen in group B.

Review ID: 11676811
Abstractive Summary: four of 56 HCTs ( 79 percent) found the therapy better than the control regimen. but only 10 of 50 RCTs ( 20 percent) agreed Adjustment of the outcomes of the HCTs for prognostic factors.

Review ID: 12719681
Abstractive Summary: one hundred and thirty eight patients did not have a home nebuliser before admission Logistic regression analysis of the three SGRQ subscales ( Symptom, Impact and Activity ) all three subscales were significantly related to hospital readmission and that Impact scores were related to nebuliser provision. 111 of those not included in the study because they refused the interview or died before discharge.

Review ID: 26830881
Ab

***SciBERT and GPT4***

Here are abstractive summaries for each biomedical review:

Review ID: 16801507
This review evaluates the effects of vitamin K2 on bone health, particularly in preventing bone loss induced by glucocorticoids and in postmenopausal women. The study involves various markers of bone metabolism and the incidence of vertebral fractures. The effects of different medications, including hormone replacement therapy, etidronate, calcitonin, and vitamin K, on bone mineral density and fracture incidence in postmenopausal women with osteoporosis are assessed. Additionally, it explores the role of vitamin K in preventing bone mineral density loss and fractures in elderly patients with Parkinson's disease, highlighting its potential in the treatment of osteoporosis.

Review ID: 11676811
This review compares the use of randomized controls (RCTs) and historical controls (HCTs) in clinical trials. It found that HCTs more often reported therapies as better than the control regimen compared to RCTs. The review also discusses the effectiveness of various interventions in preventing or treating low back pain in different settings, highlighting the mixed results and the need for further research. The findings underscore the importance of methodological rigor in clinical trials and suggest that RCTs may be more reliable for evaluating new therapies.

Review ID: 12719681
This review focuses on the impact of domiciliary oxygen therapy, self-directed exercise, and rehabilitation in patients with COPD. It examines quality of life, lung function, and hospital readmission rates. Additionally, it explores the long-term effects of pulmonary rehabilitation programs, including their impact on walking distance, muscle force, and overall quality of life. The review also investigates the perceptions of asthmatics to changes in their condition and the effectiveness of quality-of-life measures in clinical trials.

Review ID: 26830881
This review discusses long-term studies on the treatment of panic disorder and generalized anxiety disorder (GAD) with medications like clonazepam, paroxetine, and venlafaxine. It highlights the sustained efficacy of these medications over extended periods and their impact on symptoms like sleep disturbances. The review also explores pharmacogenetics in treatment response, the effectiveness of cognitive-behavioral therapy (CBT), and tapering off high-potency benzodiazepines in panic disorder treatment. Safety, adverse events, and relapse rates are key considerations.

Review ID: 28611377
This review compares outcomes in patients with persistent and paroxysmal atrial fibrillation (AF), particularly in relation to stroke risk and survival rates. It investigates the risk of early death post-ischemic stroke in AF patients, the effects of atrial flutter, and mortality associated with different anticoagulation treatments. The review also explores the correlation between AF and myocardial infarction, highlighting the public health burden of AF and the need for holistic management, including cardiovascular risk prevention.

#### prompted to pay attention to the evidence direction 


Review ID: 16801507
This review examines the role of vitamin K2 in bone health. It finds that vitamin K2 may help in preventing bone loss caused by glucocorticoids and could be beneficial in treating osteoporosis, particularly in postmenopausal women. The study reports a decrease in bone mineral density with various treatments, but notes that vitamin K2 therapy may effectively prevent new fractures and reduce postmenopausal spinal bone loss. The results suggest that early postmenopausal intervention with vitamin K2 could be beneficial in maintaining bone health.

Review ID: 11676811
The review compares randomized control trials (RCTs) and historical control trials (HCTs) in assessing clinical therapies. It reveals that HCTs more frequently report positive outcomes compared to RCTs. In evaluating low back pain interventions, the review finds mixed results on their effectiveness, with some interventions showing benefit and others showing no significant impact. The findings highlight the variability in effectiveness of different interventions and underscore the importance of robust trial designs in determining the true efficacy of therapies.

Review ID: 12719681
This review focuses on treatments for COPD, particularly examining the effectiveness of domiciliary oxygen therapy, exercise, and rehabilitation. The study reports that while rehabilitation may improve quality of life, it doesn't necessarily correlate with improvements in lung function and exercise tolerance. Self-directed exercise is noted to improve physical functioning compared to usual care. The review also observes that long-term pulmonary rehabilitation programs can lead to sustained improvements in walking distance and quality of life for COPD patients.

Review ID: 26830881
The review discusses long-term treatment of panic disorder and generalized anxiety disorder (GAD) with clonazepam, paroxetine, and venlafaxine. It finds that these medications maintain their efficacy over extended periods, with particular improvements in anxiety-related sleep disturbances. The review also notes the effective role of cognitive-behavioral therapy (CBT) in treating these disorders and suggests that tapering off benzodiazepines like clonazepam can be achieved safely in patients with panic disorder. The study underscores the importance of considering safety and the potential for relapse in long-term treatment plans.

Review ID: 28611377
This review compares the risks associated with persistent and paroxysmal atrial fibrillation (AF), particularly in the context of stroke and survival rates. It finds that patients with persistent AF have a higher risk of thrombo-embolic events and worse survival compared to those with paroxysmal AF. The study also reports that AF patients have a higher risk of early death following an ischemic stroke, largely due to older age and larger infarcts. The effectiveness of anticoagulation treatments like dabigatran in reducing vascular mortality is noted, along with the association of AF with an increased risk of myocardial infarction. The review highlights the significant public health burden of AF and the necessity for comprehensive management strategies.